<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>1D ConvNet on Sentiment Analysis</H1></u></center>

# First run this from terminal

In [801]:
# import nltk
# nltk.download('stopwords')

In [802]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import plotly.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

In [803]:
import time
start_time = time.time()
import pandas as pd
import numpy as np

In [804]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

In [805]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, Embedding, LSTM, CuDNNLSTM, Dropout, Flatten, Input
from keras.datasets import imdb

In [806]:
num_words = 20000

In [807]:
# def load_data(path='imdb.npz', num_words=None, skip_top=0,
#               maxlen=None, seed=113,
#               start_char=1, oov_char=2, index_from=3, **kwargs):

#     with np.load(path) as f:
#         x_train, labels_train = f['x_train'], f['y_train']
#         x_test, labels_test = f['x_test'], f['y_test']

#     np.random.seed(seed)
#     indices = np.arange(len(x_train))
#     np.random.shuffle(indices)
#     x_train = x_train[indices]
#     labels_train = labels_train[indices]

#     indices = np.arange(len(x_test))
#     np.random.shuffle(indices)
#     x_test = x_test[indices]
#     labels_test = labels_test[indices]

#     xs = np.concatenate([x_train, x_test])
#     labels = np.concatenate([labels_train, labels_test])

#     if start_char is not None:
#         xs = [[start_char] + [w + index_from for w in x] for x in xs]
#     elif index_from:
#         xs = [[w + index_from for w in x] for x in xs]

#     if maxlen:
#         xs, labels = _remove_long_seq(maxlen, xs, labels)
#         if not xs:
#             raise ValueError('After filtering for sequences shorter than maxlen=' +
#                              str(maxlen) + ', no sequence was kept. '
#                              'Increase maxlen.')
#     if not num_words:
#         num_words = max([max(x) for x in xs])

#     if oov_char is not None:
#         xs = [[w if (skip_top <= w < num_words) else oov_char for w in x]
#               for x in xs]
#     else:
#         xs = [[w for w in x if skip_top <= w < num_words]
#               for x in xs]

#     idx = len(x_train)
#     x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
#     x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])

#     return (x_train, y_train), (x_test, y_test)

In [808]:
#(train_data, train_labels), (test_data, test_labels) = load_data(path='imdb.npz',num_words=10000)

In [809]:
# (X_train, y_train), (X_test, y_test) = load_data(path='imdb.npz',num_words=10000)

In [810]:
df = pd.read_csv('Reviews_2.csv')

In [811]:
df.columns

Index(['HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Summary',
       'Text'],
      dtype='object')

In [812]:
df = df[['Text','Score']]

In [813]:
df.head()

,Text,Score
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5


In [814]:
len(df)

568454

In [815]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 2 columns):
Text     568454 non-null object
Score    568454 non-null int64
dtypes: int64(1), object(1)
memory usage: 8.7+ MB


In [816]:
df.Score.value_counts()

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

In [817]:
df['Score'].value_counts().sort_values(ascending=False).iplot(kind='bar', yTitle='Number of Reviews', 
                                                                title='Number reviews in each rating')

## Down-sampling the data

In [818]:
# # Using Ramdom Under Sampling (RUS) technique
# X = df.iloc[:, :1].values
# y = df.loc[:, 'Score'].values

# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(return_indices=True)
# X_rus, y_rus, id_rus = rus.fit_sample(X, y)

In [819]:
# X_rus[:,0]

In [820]:
#y_rus[:,1]

In [821]:
# X_rus.shape

In [822]:
# X_rus[10]

In [823]:
# len(X)

In [824]:
# len(X_rus)

In [825]:
# id_rus

In [826]:
# X_rus = X_rus[:,0]

In [827]:
# X_rus.shape

In [828]:
# y_rus

In [829]:
# y_rus.shape

In [830]:
##y_rus = y_rus[:,0]

In [831]:
# df = pd.DataFrame({'Text':X_rus,'Score':y_rus})

## df = pd.DataFrame({'x':x, 'y':y})

In [832]:
# df['Text'][1]

In [833]:
def print_plot(index):
    example = df[df.index == index][['Text', 'Score']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Score:', example[1])

In [834]:
print_plot(10)

I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!
Score: 5


In [835]:
print_plot(100)

The mouth says, "How do I love thee, let me count the ways..."<br />If you like apple products a must have item.  The only draw back, shipping cost.  These are very heavy.
Score: 5


In [836]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['Text'] = df['Text'].apply(clean_text)

In [837]:
df['Text'] = df['Text'].str.replace('\d+', '')

In [838]:
print_plot(10)

dont know cactus tequila unique combination ingredients flavour hot sauce makes one kind picked bottle trip brought back home us totally blown away realized simply couldnt find anywhere city bummedbr br magic internet case sauce ecstatic itbr br love hot saucei mean really love hot sauce dont want sauce tastelessly burns throat grab bottle tequila picante gourmet de inclan realize taste never want use saucebr br thank personal incredible service
Score: 5


In [839]:
print_plot(100)

mouth says love thee let count waysbr like apple products must item draw back shipping cost heavy
Score: 5


# ==================================

## Test- only first 5000 observations

In [840]:
len(df)

568454

In [841]:
df.head()

,Text,Score
0,bought several vitality canned dog food produc...,5
1,product arrived labeled jumbo salted peanutsth...,1
2,confection around centuries light pillowy citr...,4
3,looking secret ingredient robitussin believe f...,2
4,great taffy great price wide assortment yummy ...,5


# Shuffle the dataframe index, inplace

In [842]:
# df = df.sample(frac=1).reset_index(drop=True)

In [843]:
#df = df[:5000]

In [844]:
df.head()

,Text,Score
0,bought several vitality canned dog food produc...,5
1,product arrived labeled jumbo salted peanutsth...,1
2,confection around centuries light pillowy citr...,4
3,looking secret ingredient robitussin believe f...,2
4,great taffy great price wide assortment yummy ...,5


In [845]:
df['Score'].value_counts()

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

# ====================================

In [846]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 50

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 199055 unique tokens.


In [847]:
X = tokenizer.texts_to_sequences(df['Text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (568454, 250)


In [848]:
Y = pd.get_dummies(df['Score']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (568454, 5)


In [849]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(511608, 250) (511608, 5)
(56846, 250) (56846, 5)


# finished data preprocessing

# ====================================================================

# Glove embedding
# https://medium.com/@sabber/classifying-yelp-review-comments-using-cnn-lstm-and-pre-trained-glove-word-embeddings-part-3-53fcea9a17fa


In [850]:
embeddings_index = dict()
# f = open('./glove.6B.100d.txt')
f = open('./glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [851]:
# embedding_matrix = np.zeros((MAX_NB_WORDS, 100))
embedding_matrix = np.zeros((MAX_NB_WORDS, 50))
for word, index in tokenizer.word_index.items():
    if index > MAX_NB_WORDS - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [852]:
X.shape[1]

250

In [853]:
print(len(X_train), 'train_data')
print(len(X_test), 'test_data')

511608 train_data
56846 test_data


In [854]:
#(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

In [855]:
max_len = 250
embedding_size = 50
batch_size = 128

## Sequences per time

In [856]:
print(X_train[2])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [857]:
len(X_train[1])

250

In [858]:
pad  =  'post' #'pre'
#Convert our lists to equal length sequences
X_train_pad = pad_sequences(X_train, maxlen=max_len, padding=pad, truncating=pad)
X_test_pad = pad_sequences(X_test, maxlen=max_len, padding=pad, truncating=pad)

In [859]:
X_train_pad.shape

(511608, 250)

In [860]:
X_train_pad[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [861]:
X_train_pad[4499]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [862]:
X.shape[1]

250

In [863]:
X_train_pad.shape[1]

250

In [ ]:
epochs = 30
batch_size = 64

## create model
model_glove = Sequential()
model_glove.add(Embedding(MAX_NB_WORDS, 50, input_length=X_train_pad.shape[1], weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model_glove.add(MaxPooling1D(pool_size=5))
model_glove.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model_glove.add(MaxPooling1D(pool_size=5))
model_glove.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model_glove.add(MaxPooling1D(pool_size=5))
# model0.add(Dropout(0.2))
model_glove.add(Flatten())
model_glove.add(Dense(250, activation='relu'))
model_glove.add(Dense(5, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data

# model_glove.fit(data, np.array(labels), validation_split=0.4, epochs = 3)

#%%time
model_glove.fit(X_train_pad, Y_train, validation_split=0.05, epochs=epochs, batch_size=batch_size)

# if you want to show the train-validation graphs, uncomment this

# import matplotlib.pyplot as plt
# plt.style.use('ggplot')

# def plot_history(history):
#     acc = history.history['acc']
#     val_acc = history.history['val_acc']
#     loss = history.history['loss']
#     val_loss = history.history['val_loss']
#     x = range(1, len(acc) + 1)

#     plt.figure(figsize=(12, 5))
#     plt.subplot(1, 2, 1)
#     plt.plot(x, acc, 'b', label='Training acc')
#     plt.plot(x, val_acc, 'r', label='Validation acc')
#     plt.title('Training and validation accuracy')
#     plt.legend()
#     plt.subplot(1, 2, 2)
#     plt.plot(x, loss, 'b', label='Training loss')
#     plt.plot(x, val_loss, 'r', label='Validation loss')
#     plt.title('Training and validation loss')
#     plt.legend()

# history = model_glove.fit(X_train_pad, Y_train,
#                     epochs=epochs,
#                     verbose=False,
#                     validation_split=0.2222,
#                     batch_size=batch_size)
# loss, accuracy = model0.evaluate(X_train_pad, Y_train, verbose=False)
# print("Training Accuracy: {:.4f}".format(accuracy))
# loss, accuracy = model0.evaluate(X_test_pad, Y_test, verbose=False)
# print("Testing Accuracy:  {:.4f}".format(accuracy))
# plot_history(history)


Train on 486027 samples, validate on 25581 samples
Epoch 1/30
486027/486027 [==============================] - 269s 554us/step - loss: 0.9011 - acc: 0.6767 - val_loss: 0.8325 - val_acc: 0.6973
Epoch 2/30
486027/486027 [==============================] - 261s 537us/step - loss: 0.8365 - acc: 0.6957 - val_loss: 0.8044 - val_acc: 0.7088
Epoch 3/30
  7488/486027 [..............................] - ETA: 4:14 - loss: 0.8132 - acc: 0.7007

## Testing

In [ ]:
#%%time
eval_ = model_glove.evaluate(X_test_pad, Y_test)

In [ ]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

In [ ]:
# history = model.fit(X_train, y_train,
#                     epochs=10,
#                     verbose=False,
#                     validation_data=(X_test, y_test),
#                     batch_size=10)
# loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
# print("Training Accuracy: {:.4f}".format(accuracy))
# loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
# print("Testing Accuracy:  {:.4f}".format(accuracy))
# plot_history(history)

# ===============================================

# Now show the F-1 score and Accuracy

In [ ]:
accr = model_glove.evaluate(X_test_pad,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))



In [ ]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
def decode(datum):
    return np.argmax(datum)

In [ ]:
# for i in range(Y_test.shape[0]):
#     datum = Y_test[i]
#     print('index: %d' % i)
#     print('encoded datum: %s' % datum)
#     decoded_datum = decode(Y_test[i])
#     print('decoded datum: %s' % decoded_datum)
#     print()

In [ ]:
Y_test = np.array([decode(i) for i in Y_test])

In [ ]:
yhat_probs = model_glove.predict(X_test_pad, verbose=0)
# predict crisp classes for test set
yhat_classes = model_glove.predict_classes(X_test_pad, verbose=0)

In [907]:
yhat_classes

array([4, 4, 2, ..., 0, 0, 1])

In [908]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Y_test, yhat_classes)
print('Accuracy: %f' % accuracy)

Accuracy: 0.754213


In [909]:
f1_score(Y_test, yhat_classes, average='macro') 

0.5586983066673321

In [910]:
f1_score(Y_test, yhat_classes, average='micro') 

0.7542131372480034

In [911]:
f1_score(Y_test, yhat_classes, average=None) 

array([0.66847217, 0.36877524, 0.41813668, 0.46162496, 0.87648248])

In [912]:
f1_score(Y_test, yhat_classes, average='weighted') 

0.7389385666164378

In [913]:
unique_elements, counts_elements = np.unique(Y_test, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[    0     1     2     3     4]
 [ 5179  2904  4267  7982 36514]]


In [914]:
unique_elements, counts_elements = np.unique(yhat_classes, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[    0     1     2     3     4]
 [ 5529  1554  3386  6246 40131]]


In [915]:
# confusion matrix
matrix = confusion_matrix(Y_test, yhat_classes)
print(matrix)

[[ 3579   272   243   205   880]
 [  755   822   436   229   662]
 [  430   255  1600   771  1211]
 [  221   104   584  3284  3789]
 [  544   101   523  1757 33589]]


# ==================================

## Model 1:

In [ ]:
# model1 = Sequential()

In [ ]:
# model1.add(Embedding(input_dim=num_words,
#                     output_dim=embedding_size,
#                     input_length=max_len,
#                     name='layer_embedding'))

In [ ]:
# model1.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))

In [ ]:
# model1.add(MaxPooling1D(pool_size=3))

In [ ]:
# model1.add(CuDNNLSTM(128))

In [ ]:
# model1.add(Dropout(0.5))

In [ ]:
# model1.add(Dense(1, activation='sigmoid', name='classification'))

## Compiling the model:

In [ ]:
# model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# model1.summary()

## Training the model:

In [ ]:
# %%time
# model1.fit(X_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

## Testing the model:

In [ ]:
# %%time
# eval_ = model1.evaluate(X_test_pad, y_test)

In [ ]:
# print("Loss: {0:.5}".format(eval_[0]))
# print("Accuracy: {0:.2%}".format(eval_[1]))

## Conv1D model with multiple kernel sizes:

In [ ]:
# from keras.layers import concatenate

In [ ]:
# conv = []
# filter_sizes = [3,4,5]

In [ ]:
# embedding_layer = Embedding(input_dim=num_words,
#                     output_dim=embedding_size,
#                     input_length=max_len,
#                     name='layer_embedding')

In [ ]:
# seq_input = Input(shape=(max_len,), dtype='int32')
# embed_seq = embedding_layer(seq_input)

# for f in filter_sizes:
#     _conv = Conv1D(filters=128,kernel_size=f,activation='relu')(embed_seq)
#     _pool = MaxPooling1D(5)(_conv)
#     conv.append(_pool)

In [ ]:
# _concat = concatenate(conv, axis=1)
# _conv1= Conv1D(128, 5, activation='relu')(_concat)
# _pool1 = MaxPooling1D(5)(_conv1)
# _pool1 = Dropout(0.5)(_pool1)
# _conv2 = Conv1D(128, 5, activation='relu')(_pool1)
# _pool2 = MaxPooling1D(5)(_conv2)
# _flat = Flatten()(_pool2)
# _flat = Dropout(0.5)(_flat)
# _dense = Dense(128, activation='relu')(_flat)
# preds = Dense(1, activation='sigmoid')(_dense)

In [ ]:
# model = Model(seq_input, preds)

In [ ]:
# model.summary()

In [ ]:
# model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [ ]:
# %%time
# model.fit(X_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

In [ ]:
# %%time
# eval_ = model.evaluate(X_test_pad, y_test)

In [ ]:
# print("Loss: {0:.5}".format(eval_[0]))
# print("Accuracy: {0:.2%}".format(eval_[1]))

In [ ]:
print("Total time used....")
print("--- %s seconds ---" % (time.time() - start_time))

## Reference:

https://keras.io/layers/convolutional/